In [1]:
import bpy

In [2]:
import cProfile
from IPython.display import Image
from pstats import Stats

# pip install gprof2dot

def profile(exec_code):
    cProfile.run(exec_code, filename='/tmp/cprof.pstats')
    !gprof2dot -f pstats /tmp/cprof.pstats | dot -Tpng -o /tmp/output.png
    return Image(filename='/tmp/output.png')

In [3]:
list(bpy.data.objects)

[bpy.data.objects['Camera'],
 bpy.data.objects['Cube'],
 bpy.data.objects['Light']]

In [4]:
bpy.context.scene.render.filepath = 'C:/Users/orenm/Desktop/test.png'
bpy.context.scene.render.image_settings.file_format='PNG'

In [9]:
bpy.ops.render.render(write_still=True, use_viewport = True)

{'FINISHED'}

can i do the render stuff actuallz inside blender? so the viewport is open and i just need to sample it?

looks promising:
https://blenderartists.org/t/accessing-viewport-render-result-pixels-from-script/1173506/7

ask this guy about it?
https://www.youtube.com/@CGPython/videos

In [ ]:
https://stackoverflow.com/questions/68571715/in-blender-how-do-i-programmatically-display-a-viewport-rendering-in-a-window-j
https://devtalk.blender.org/t/how-to-realtime-render-get-datauri-of-the-blender-viewport-in-full-resolution-1920-1080/5504
https://blenderartists.org/t/way-to-track-the-viewport-point-of-view/1393853
https://devtalk.blender.org/t/how-to-realtime-render-get-datauri-of-the-blender-viewport-in-full-resolution-1920-1080/5504/7